<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.31.1
    Uninstalling protobuf-6.31.1:
      Successfully uninstalled protobuf-6.31.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.61
    Uninstalling yfinance-0.2.61:
      Successfully uninstalled yfinance-0.2.61
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))
  print('-------------------')
  print('Est LTT: ',  format_amt(estimate_lt))
  print('Est LTT PnL: '+ '{} ({}%)'.format(format_amt(est_lt_pnl_amount), est_lt_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')

Run date time (IST): 2025-06-03 11:45:39
-------------------
qualified stocks: 93
with latest results: 32
still star stocks: 18
-------------------
Initial Investment:  1.17 C
CY Investment:  1.44 C
Reserve:  4.24 L
Current:  1.32 C
-------------------
Today PnL: -25.60 K (-0.19%)
Current PnL: -16.12 L (-11.23%)
CY Booked + Current PnL: -11.65 L (-8.12%)
-------------------
Total profit:  4.30 L
Total loss:  -20.42 L
-------------------
Total Booked + Current PnL: 15.33 L (13.15%)
Total Booked PnL: 31.45 L (26.99%)
Curr Year Booked PnL: 4.47 L (3.38%)
Prev Year Booked PnL: 26.98 L (23.15%)
Est FTT:  2.11 C
Est FTT PnL: 78.32 L (59.21%)
-------------------
Est LTT:  2.73 C
Est LTT PnL: 1.40 C (106.18%)
Deployed:  1.17 C
Current:  1.32 C
CAGR/XIRR %: 9.05%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'RSP', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,PIDILITIND,2504.06,3202.0,-16.34,X-LC,37.63,92655.0,17533.0,3400.0,0.27,23.34,3.67,27.87,5.0,5.16,0.72,16.20,X40,NTT,CHEMICALS
49,LICI,785.05,983.0,-3.65,H-LC,88.17,236642.0,41950.0,7147.0,-0.85,21.55,3.02,25.21,31.0,5.87,1.85,30.86,XY25,NTT,INSURANCE
4,ADANIPORTS,1103.69,1583.0,2.14,M-LC,61.29,102446.0,24084.0,9948.0,-1.71,30.73,9.71,43.43,70.0,2.42,0.80,37.02,XY24,NTT,MISC
85,UNIONBANK,123.87,163.0,-0.31,M-LC,91.40,173245.0,32405.0,12093.0,-0.52,23.01,6.98,31.59,78.0,2.68,1.35,50.49,XY24,NTT,BANKS
62,RELIANCE,1291.83,1526.0,3.85,X-LC,17.20,154093.0,13284.0,12235.0,-0.51,9.43,7.94,18.13,19.0,1.09,1.20,21.67,XY25,NTT,REFINERIES


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
67,SATIN,199.77,274.00,-21.16,H-SC,12.90,156734.0,-39240.0,112065.0,3.80,-20.02,71.50,37.16,124.0,-0.35,1.22,17.05,XY24,NTT,FINANCE
47,LALPATHLAB,2804.19,3545.00,-32.60,H-SC,40.32,155243.0,1013.0,39727.0,2.04,0.66,25.59,26.42,114.0,0.03,1.21,22.17,X40N,NTT,HEALTHCARE
38,INDIAMART,2327.09,4911.36,-51.84,H-SC,56.45,125663.0,2327.0,134635.0,2.02,1.89,107.14,111.05,108.0,0.02,0.98,23.13,AR,ATH,MISC
70,SHALBY,261.39,327.00,852.15,M-SC,26.88,129556.0,-51587.0,97050.0,1.80,-28.48,74.91,25.10,206.0,-0.53,1.01,3.69,XY24,NTT,HEALTHCARE
87,VALIANTORG,512.64,838.00,-385.09,H-SC,100.00,121316.0,-11970.0,96568.0,1.57,-8.98,79.60,63.47,131.0,-0.12,0.95,103.36,XR,NTT,CHEMICALS


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
37,ICICIPRULI,600.83,791.05,-9.87,H-MC,52.69,145578.0,9790.0,33206.0,-3.01,7.21,22.81,31.66,105.0,0.29,1.14,20.04,X40,ATH,INSURANCE
19,COALINDIA,406.65,512.15,3.82,L-LC,38.71,145280.0,-5180.0,44209.0,-1.76,-3.44,30.43,25.94,145.0,-0.12,1.13,10.86,XY25,ATH,MINING
4,ADANIPORTS,1103.69,1583.00,2.14,M-LC,61.29,102446.0,24084.0,9948.0,-1.71,30.73,9.71,43.43,70.0,2.42,0.80,37.02,XY24,NTT,MISC
20,COFFEEDAY,59.14,80.00,-59.76,L-SC,8.60,58656.0,-54893.0,94946.0,-1.42,-48.34,161.87,35.27,239.0,-0.58,0.46,43.56,XR,NTT,HOTELS
59,RAJESHEXPO,517.65,583.00,1828.03,M-SC,46.24,52595.0,-84582.0,101903.0,-1.41,-61.66,193.75,12.62,208.0,-0.83,0.41,30.80,X40N,NTT,JEWELLERY


In [12]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
41,INFY,1450.31,2275.00,-13.50,X-LC,47.31,246037.0,15438.0,115687.0,-0.41,6.69,47.02,56.86,1.0,0.13,1.92,13.88,X40,BTT,IT
80,TCS,3841.06,4998.00,-18.09,X-LC,19.35,236366.0,-28667.0,108492.0,-0.70,-10.82,45.90,30.12,2.0,-0.26,1.85,6.01,X40,BTT,IT
50,LTIM,5564.16,7197.33,-5.69,X-LC,71.51,181368.0,-18942.0,77734.0,-0.68,-9.46,42.86,29.35,3.0,-0.24,1.42,25.30,X200,ATH,IT
56,PIDILITIND,2504.06,3202.00,-16.34,X-LC,37.63,92655.0,17533.0,3400.0,0.27,23.34,3.67,27.87,5.0,5.16,0.72,16.20,X40,NTT,CHEMICALS
6,ASIANPAINT,2961.56,4250.00,-23.00,X-LC,4.30,153231.0,-48155.0,135763.0,-0.59,-23.91,88.60,43.51,6.0,-0.35,1.20,5.69,X40,BTT,PAINTS


In [13]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,PFIZER,4245.50,6318.3,13.55,H-SC,98.92,131226.0,33580.0,14094.0,1.46,34.39,10.74,48.82,134.0,2.38,1.02,45.78,X40,ATH,PHARMA
85,UNIONBANK,123.87,163.0,-0.31,M-LC,91.40,173245.0,32405.0,12093.0,-0.52,23.01,6.98,31.59,78.0,2.68,1.35,50.49,XY24,NTT,BANKS
4,ADANIPORTS,1103.69,1583.0,2.14,M-LC,61.29,102446.0,24084.0,9948.0,-1.71,30.73,9.71,43.43,70.0,2.42,0.80,37.02,XY24,NTT,MISC
49,LICI,785.05,983.0,-3.65,H-LC,88.17,236642.0,41950.0,7147.0,-0.85,21.55,3.02,25.21,31.0,5.87,1.85,30.86,XY25,NTT,INSURANCE
56,PIDILITIND,2504.06,3202.0,-16.34,X-LC,37.63,92655.0,17533.0,3400.0,0.27,23.34,3.67,27.87,5.0,5.16,0.72,16.20,X40,NTT,CHEMICALS


In [14]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
65,SAIL,130.64,228.00,99.34,M-MC,84.95,227132.0,2170.0,165488.0,0.93,0.96,72.86,74.53,170.0,0.01,1.77,31.87,XY24,BTT,STEEL
51,LUXIND,1448.19,2442.00,38.95,M-SC,67.74,144115.0,6537.0,87881.0,-0.50,4.75,60.98,68.62,183.0,0.07,1.13,24.94,X40N,NTT,TEXTILES
55,PFIZER,4245.50,6318.30,13.55,H-SC,98.92,131226.0,33580.0,14094.0,1.46,34.39,10.74,48.82,134.0,2.38,1.02,45.78,X40,ATH,PHARMA
28,GILLETTE,8109.44,10330.69,10.50,H-SC,90.32,162920.0,25060.0,12708.0,1.21,18.18,7.80,27.39,116.0,1.97,1.27,36.31,X40,ATH,FMCG
57,POLYCAB,5005.95,7541.35,5.53,H-MC,36.56,100206.0,15105.0,27998.0,-0.54,17.75,27.94,50.65,90.0,0.54,0.78,25.96,X40N,ATH,CABLES


In [15]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
17,CAMS,3643.00,5250.99,3.78,H-SC,65.59,116718.0,14714.0,30312.0,0.71,14.42,25.97,44.14,111.0,0.49,0.91,33.01,XR,ATH,MISC
11,BANKINDIA,113.49,190.00,-27.01,M-MC,62.37,142068.0,12916.0,74145.0,-0.33,10.00,52.19,67.42,156.0,0.17,1.11,38.17,XR,NTT,BANKS
38,INDIAMART,2327.09,4911.36,-51.84,H-SC,56.45,125663.0,2327.0,134635.0,2.02,1.89,107.14,111.05,108.0,0.02,0.98,23.13,AR,ATH,MISC
92,WIPRO,243.46,420.00,-11.07,M-LC,33.33,152532.0,1587.0,107871.0,-0.67,1.05,70.72,72.51,68.0,0.01,1.19,14.53,XR,NTT,IT
1,AARTIIND,487.04,919.00,38.08,M-SC,54.84,79333.0,-2003.0,74137.0,0.93,-2.46,93.45,88.69,205.0,-0.03,0.62,33.78,XR,NTT,CHEMICALS


In [16]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
52,MASFIN,326.60,399.5,-22.31,H-SC,68.82,88200.0,-9780.0,31646.0,-0.03,-9.98,35.88,22.32,125.0,-0.31,0.69,27.92,XR,ATH,FINANCE
4,ADANIPORTS,1103.69,1583.0,2.14,M-LC,61.29,102446.0,24084.0,9948.0,-1.71,30.73,9.71,43.43,70.0,2.42,0.80,37.02,XY24,NTT,MISC
84,UJJIVANSFB,52.77,60.0,36.52,M-SC,56.45,118692.0,-23787.0,43311.0,-1.19,-16.70,36.49,13.70,221.0,-0.55,0.93,40.09,X40N,NTT,BANKS
87,VALIANTORG,512.64,838.0,-385.09,H-SC,100.00,121316.0,-11970.0,96568.0,1.57,-8.98,79.60,63.47,131.0,-0.12,0.95,103.36,XR,NTT,CHEMICALS
55,PFIZER,4245.50,6318.3,13.55,H-SC,98.92,131226.0,33580.0,14094.0,1.46,34.39,10.74,48.82,134.0,2.38,1.02,45.78,X40,ATH,PHARMA


In [17]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
87,VALIANTORG,512.64,838.00,-385.09,H-SC,100.00,121316.0,-11970.0,96568.0,1.57,-8.98,79.60,63.47,131.0,-0.12,0.95,103.36,XR,NTT,CHEMICALS
55,PFIZER,4245.50,6318.30,13.55,H-SC,98.92,131226.0,33580.0,14094.0,1.46,34.39,10.74,48.82,134.0,2.38,1.02,45.78,X40,ATH,PHARMA
7,ASIANTILES,75.41,137.00,6522.22,M-SC,97.85,72637.0,-21173.0,97791.0,-1.00,-22.57,134.63,81.67,214.0,-0.22,0.57,40.66,XR,NTT,CERAMICS
74,STARHEALTH,551.56,761.00,5.39,M-SC,96.77,172389.0,-27827.0,103847.0,0.17,-13.90,60.24,37.97,196.0,-0.27,1.35,38.41,XY24,NTT,INSURANCE
77,TANLA,917.30,1963.11,-33.79,H-SC,95.70,162003.0,-77412.0,350364.0,-1.15,-32.33,216.27,114.01,113.0,-0.22,1.27,48.58,AR,ATH,IT


In [18]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,18.81
1,25,41.19
2,50,70.02


In [19]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24    32.19
XY25    14.37
X40     14.26
X40N    13.57
AR      12.44
XR      10.62
X200     1.42
SR       1.16
Name: CurrAlloc%, dtype: float64

In [20]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    21.73
M-SC    17.84
X-LC    17.29
H-MC     8.66
M-LC     7.82
H-LC     7.28
X-MC     5.66
M-MC     5.62
X-SC     2.81
L-SC     2.33
L-LC     1.13
Name: CurrAlloc%, dtype: float64

In [21]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,13.45,-0.61,32.39
BANKS,9.93,-10.14,60.77
IT,9.88,-18.68,83.54
MISC,6.52,-29.54,86.84
FINANCE,5.99,-24.55,83.69
INSURANCE,5.42,3.69,25.18
PAINTS,5.39,-19.34,54.66
ELECTRICAL,5.00,-5.06,51.16
AUTO,4.40,-11.06,59.57


In [22]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2851463.0,26
X40N,1210800.0,17
AR,1176180.0,12
XR,1094254.0,14
X40,763716.0,11
XY25,461510.0,10
SR,196508.0,2
X200,77734.0,1


In [23]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2234200.0,21
M-SC,2033716.0,21
X-LC,798962.0,12
M-MC,547800.0,5
H-MC,518181.0,8
X-SC,349234.0,4
X-MC,308567.0,5
M-LC,303057.0,6
L-SC,299748.0,4


In [24]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                 
H-SC       XY24      933087.0      6
M-SC       XY24      748520.0      6
H-SC       AR        544592.0      3
           X40N      497254.0      6
M-SC       X40N      469756.0      6
X-LC       X40       450534.0      5
M-MC       XY24      391273.0      3
M-SC       XR        293058.0      4
H-SC       XR        232465.0      4
M-SC       X40       223024.0      2
H-LC       AR        185663.0      3
M-LC       XY24      182098.0      4
H-MC       XY24      174576.0      2
L-SC       XR        157148.0      2
X-LC       AR        144541.0      2
X-MC       XY24      138970.0      2
X-LC       XY25      126153.0      4
X-SC       X40N      121142.0      2
M-SC       XY25      120477.0      1
X-SC       XR        116334.0      1
H-MC       XR        113233.0      1
X-SC       SR        111758.0      1
M-LC       XR        107871.0      1
X-MC       X40N       94650.0      2
M-SC       AR         94131.0      1
           SR         84750.0      1
M-MC       AR         82382.0      1
L-SC       XY24       81258.0      1
X-LC       X200       77734.0      1
H-MC       XY25       75489.0      1
X-MC       XY25       74947.0      1
M-MC       XR         74145.0      1
H-MC       AR         63529.0      1
           X40        63356.0      2
L-SC       AR         61342.0      1
L-LC       XY25       44209.0      1
H-MC       X40N       27998.0      1
H-SC       X40        26802.0      2
M-LC       XY25       13088.0      1
H-LC       XY25        7147.0      1

In [25]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 31.0 seconds
